In [43]:
from pyspark.sql import SparkSession
import getpass
username=getpass.getuser()
spark=SparkSession.\
    builder.\
    appName("spark"). \
    config('spark.ui.port','0').\
    config("spark.sql.warehouse.dir",f"/user/{username}/warehouse").\
    enableHiveSupport().\
    master('yarn').\
    getOrCreate()

In [2]:
order_schema = 'order_id long, order_date date, customer_id long, order_status string'

In [3]:
df = spark.read \
.format("csv") \
.schema(order_schema) \
.load("source_file_path")

In [4]:
df.show(5)

+--------+----------+-----------+---------------+
|order_id|order_date|customer_id|   order_status|
+--------+----------+-----------+---------------+
|       1|2013-07-25|      11599|         CLOSED|
|       2|2013-07-25|        256|PENDING_PAYMENT|
|       3|2013-07-25|      12111|       COMPLETE|
|       4|2013-07-25|       8827|         CLOSED|
|       5|2013-07-25|      11318|       COMPLETE|
+--------+----------+-----------+---------------+
only showing top 5 rows



In [5]:
df.select("*").show()

+--------+----------+-----------+---------------+
|order_id|order_date|customer_id|   order_status|
+--------+----------+-----------+---------------+
|       1|2013-07-25|      11599|         CLOSED|
|       2|2013-07-25|        256|PENDING_PAYMENT|
|       3|2013-07-25|      12111|       COMPLETE|
|       4|2013-07-25|       8827|         CLOSED|
|       5|2013-07-25|      11318|       COMPLETE|
|       6|2013-07-25|       7130|       COMPLETE|
|       7|2013-07-25|       4530|       COMPLETE|
|       8|2013-07-25|       2911|     PROCESSING|
|       9|2013-07-25|       5657|PENDING_PAYMENT|
|      10|2013-07-25|       5648|PENDING_PAYMENT|
|      11|2013-07-25|        918| PAYMENT_REVIEW|
|      12|2013-07-25|       1837|         CLOSED|
|      13|2013-07-25|       9149|PENDING_PAYMENT|
|      14|2013-07-25|       9842|     PROCESSING|
|      15|2013-07-25|       2568|       COMPLETE|
|      16|2013-07-25|       7276|PENDING_PAYMENT|
|      17|2013-07-25|       2667|       COMPLETE|


In [6]:
df.select("order_id","order_date").show()

+--------+----------+
|order_id|order_date|
+--------+----------+
|       1|2013-07-25|
|       2|2013-07-25|
|       3|2013-07-25|
|       4|2013-07-25|
|       5|2013-07-25|
|       6|2013-07-25|
|       7|2013-07-25|
|       8|2013-07-25|
|       9|2013-07-25|
|      10|2013-07-25|
|      11|2013-07-25|
|      12|2013-07-25|
|      13|2013-07-25|
|      14|2013-07-25|
|      15|2013-07-25|
|      16|2013-07-25|
|      17|2013-07-25|
|      18|2013-07-25|
|      19|2013-07-25|
|      20|2013-07-25|
+--------+----------+
only showing top 20 rows



In [30]:
from pyspark.sql.functions import *

In [11]:
df.select("order_id",df.order_date,df['order_date']).show()

+--------+----------+----------+
|order_id|order_date|order_date|
+--------+----------+----------+
|       1|2013-07-25|2013-07-25|
|       2|2013-07-25|2013-07-25|
|       3|2013-07-25|2013-07-25|
|       4|2013-07-25|2013-07-25|
|       5|2013-07-25|2013-07-25|
|       6|2013-07-25|2013-07-25|
|       7|2013-07-25|2013-07-25|
|       8|2013-07-25|2013-07-25|
|       9|2013-07-25|2013-07-25|
|      10|2013-07-25|2013-07-25|
|      11|2013-07-25|2013-07-25|
|      12|2013-07-25|2013-07-25|
|      13|2013-07-25|2013-07-25|
|      14|2013-07-25|2013-07-25|
|      15|2013-07-25|2013-07-25|
|      16|2013-07-25|2013-07-25|
|      17|2013-07-25|2013-07-25|
|      18|2013-07-25|2013-07-25|
|      19|2013-07-25|2013-07-25|
|      20|2013-07-25|2013-07-25|
+--------+----------+----------+
only showing top 20 rows



## Agggregation

In [24]:
order_df = spark.read \
.format("csv") \
.option("inferSchema", "true") \
.option("header", "true") \
.load("source_file_path")

In [25]:
order_df.show()

+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|   536378|     null|PACK OF 60 DINOSA...|      24|01-12-2010 9.37|     0.55|     14688|United Kingdom|
|   536378|     null|PACK OF 60 PINK P...|      24|01-12-2010 9.37|     0.55|     14688|United Kingdom|
|   536378|    84991|60 TEATIME FAIRY ...|      24|01-12-2010 9.37|     0.55|     14688|United Kingdom|
|   536378|   84519A|TOMATO CHARLIE+LO...|       6|01-12-2010 9.37|     2.95|     14688|United Kingdom|
|   536378|   85183B|CHARLIE & LOLA WA...|      48|01-12-2010 9.37|     1.25|     14688|United Kingdom|
|   536378|   85071B|RED CHARLIE+LOLA ...|      96|01-12-2010 9.37|     0.38|     14688|United Kingdom|
|   536378|    21931|JUMBO STORAGE BAG...|      10|01-12-2010 9.

####simple aggregation

## grouping agg

In [26]:
#programatic 
summary_df = order_df \
.groupby("country", "invoiceno") \
.agg(sum("quantity").alias("total_quantity"), sum(expr("quantity * unitprice")).alias('invoice_value')).sort("invoiceno")

In [27]:
summary_df.show()

+--------------+---------+--------------+------------------+
|       country|invoiceno|total_quantity|     invoice_value|
+--------------+---------+--------------+------------------+
|United Kingdom|   536378|           242|192.78000000000003|
|United Kingdom|   536380|            24|              34.8|
|United Kingdom|   536381|           198|449.97999999999996|
|United Kingdom|   536382|           134|430.59999999999997|
|United Kingdom|   536384|           190|             489.6|
|United Kingdom|   536385|            53|            130.85|
|United Kingdom|   536386|           236|508.20000000000005|
|United Kingdom|   536387|          1440|           3193.92|
|United Kingdom|   536388|           108|            226.14|
|     Australia|   536389|           107|            358.25|
|United Kingdom|   536390|          1568|           1825.74|
|United Kingdom|   536392|           103|318.14000000000004|
|United Kingdom|   536393|             8|              79.6|
|United Kingdom|   53639

In [44]:
from pyspark.sql import *

## windowing agg

In [45]:
order_df = spark.read \
.format("csv") \
.option("inferSchema", "true") \
.option("header", "true") \
.load("source_file_path")

In [46]:
order_df.show()

+--------------+-------+-----------+-------------+------------+
|       country|weeknum|numinvoices|totalquantity|invoicevalue|
+--------------+-------+-----------+-------------+------------+
|         Spain|     49|          1|           67|      174.72|
|       Germany|     48|         11|         1795|     3309.75|
|     Lithuania|     48|          3|          622|     1598.06|
|       Germany|     49|         12|         1852|     4521.39|
|       Bahrain|     51|          1|           54|      205.74|
|       Iceland|     49|          1|          319|      711.79|
|         India|     51|          5|           95|      276.84|
|     Australia|     50|          2|          133|      387.95|
|         Italy|     49|          1|           -2|       -17.0|
|         India|     49|          5|         1280|      3284.1|
|         Spain|     50|          2|          400|     1049.01|
|United Kingdom|     51|        200|        28782|    75103.46|
|        Norway|     49|          1|    

In [47]:
mywindow = Window.partitionBy("country") \
.orderBy("weeknum") \
.rowsBetween(Window.unboundedPreceding, Window.currentRow)

In [48]:
result_df = order_df.withColumn("running_total", sum("invoicevalue").over(mywindow))

In [49]:
result_df.show()

+-------+-------+-----------+-------------+------------+------------------+
|country|weeknum|numinvoices|totalquantity|invoicevalue|     running_total|
+-------+-------+-----------+-------------+------------+------------------+
| Sweden|     50|          3|         3714|      2646.3|            2646.3|
|Germany|     48|         11|         1795|     3309.75|           3309.75|
|Germany|     49|         12|         1852|     4521.39|           7831.14|
|Germany|     50|         15|         1973|     5065.79|          12896.93|
|Germany|     51|          5|         1103|     1665.91|          14562.84|
| France|     48|          4|         1299|     2808.16|           2808.16|
| France|     49|          9|         2303|     4527.01|           7335.17|
| France|     50|          6|          529|      537.32|           7872.49|
| France|     51|          5|          847|     1702.87|           9575.36|
|Belgium|     48|          1|          528|       346.1|             346.1|
|Belgium|   

In [42]:
spark.stop()

## window functions

In [50]:
order_df = spark.read \
.format("csv") \
.option("inferSchema", "true") \
.option("header", "true") \
.load("source_file_path")

In [51]:
order_df.show()

+--------------+-------+-----------+-------------+------------+
|       country|weeknum|numinvoices|totalquantity|invoicevalue|
+--------------+-------+-----------+-------------+------------+
|         Spain|     49|          1|           67|      174.72|
|       Germany|     48|         11|         1795|      1600.0|
|     Lithuania|     48|          3|          622|     1598.06|
|       Germany|     49|         12|         1852|      1800.0|
|       Bahrain|     51|          1|           54|      205.74|
|       Iceland|     49|          1|          319|      711.79|
|         India|     51|          5|           95|       300.0|
|     Australia|     50|          2|          133|      387.95|
|         Italy|     49|          1|           -2|       -17.0|
|         India|     49|          5|         1280|      3284.1|
|         Spain|     50|          2|          400|     1049.01|
|United Kingdom|     51|        200|        28782|    75103.46|
|        Norway|     49|          1|    

In [ ]:
mywindow = Window.partitionBy("country") \
.orderBy("weeknum") \
.rowsBetween(Window.unboundedPreceding, Window.currentRow)